<a href="https://colab.research.google.com/github/ameyanjarlekar/Sparse-Attentive-Impainting/blob/master/Sparse_Attentive_Inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn

class AlexEnc_Dec(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexEnc_Dec, self).__init__()
        self.RELU = nn.ReLU(inplace=True)
        self.enc1a = nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2)
        # nn.init.xavier_uniform_(self.enc1a.weight)
        self.enc1 = nn.MaxPool2d(kernel_size=3, stride=2,return_indices=True)
        self.enc2a = nn.Conv2d(64, 192, kernel_size=5, padding=2)
        # nn.init.xavier_uniform_(self.enc2a.weight)
        self.enc2 = nn.MaxPool2d(kernel_size=3, stride=2,return_indices=True)
        self.enc3a = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        # nn.init.xavier_uniform_(self.enc3a.weight)
        self.enc3b = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        # nn.init.xavier_uniform_(self.enc3b.weight)
        self.enc3c = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        # nn.init.xavier_uniform_(self.enc3b.weight)
        self.enc3 =  nn.MaxPool2d(kernel_size=3, stride=2,return_indices=True)
        
        self.bridge = nn.Conv2d(256, 256,  kernel_size=3, padding=1)

        # self.lin = nn.Linear(4096,4096)
        #self.Encoder = nn.Sequential(
        #    nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
        #    nn.ReLU(inplace=True),
        #    nn.MaxPool2d(kernel_size=3, stride=2,return_indices=True),
        #    nn.Conv2d(64, 192, kernel_size=5, padding=2),
        #    nn.ReLU(inplace=True),
        #    nn.MaxPool2d(kernel_size=3, stride=2,return_indices=True),
        #    nn.Conv2d(192, 384, kernel_size=3, padding=1),
        #    nn.ReLU(inplace=True),
        #    nn.Conv2d(384, 256, kernel_size=3, padding=1),
        #    nn.ReLU(inplace=True),
        #    nn.Conv2d(256, 256, kernel_size=3, padding=1),
        #    nn.ReLU(inplace=True),
        #    nn.MaxPool2d(kernel_size=3, stride=2,return_indices=True),
        #)

        self.dec3 = nn.MaxUnpool2d(kernel_size=3, stride=2)
        self.dec3c = nn.ConvTranspose2d(256, 256, kernel_size=3, padding=1)
        # nn.init.xavier_uniform_(self.dec3c.weight)
        self.dec3b = nn.ConvTranspose2d(256, 384, kernel_size=3, padding=1)
        # nn.init.xavier_uniform_(self.dec3b.weight)
        self.dec3a = nn.ConvTranspose2d(384, 192, kernel_size=3, padding=1)
        # nn.init.xavier_uniform_(self.dec3a.weight)
        self.dec2 = nn.MaxUnpool2d(kernel_size=3, stride=2)
        self.dec2a = nn.ConvTranspose2d(192, 64, kernel_size=5, padding=2)
        # nn.init.xavier_uniform_(self.dec2a.weight)
        self.dec1 = nn.MaxUnpool2d(kernel_size=3, stride=2)
        self.dec1a = nn.ConvTranspose2d(64, 3, kernel_size=11, stride=4, padding=2)
        # nn.init.xavier_uniform_(self.dec1a.weight)

        #self.Decoder = nn.Sequential(
        #    nn.ConvTranspose2d(256, 256, kernel_size=3, padding=1),
        #    nn.ReLU(inplace=True),
        #    nn.MaxUnpool2d(kernel_size=3, stride=2),
        #    nn.ConvTranspose2d(256, 384, kernel_size=3, padding=1),
        #    nn.ReLU(inplace=True),
        #    nn.ConvTranspose2d(384, 192, kernel_size=3, padding=1),
        #    nn.ReLU(inplace=True),
        #    nn.ConvTranspose2d(192, 64, kernel_size=5, padding=2),
        #    nn.ReLU(inplace=True),
        #    nn.MaxUnpool2d(kernel_size=3, stride=2),
        #    nn.ConvTranspose2d(64, 3, kernel_size=11, stride=4, padding=2),
        #    nn.ReLU(inplace=True),
        #    nn.MaxUnpool2d(kernel_size=3, stride=2),
        #)

    def forward(self, x):
        #x = self.Encoder(x)
        #x = self.Decoder(x)
        #return x
        # print(x.shape)
        encconv1a = self.enc1a(x)             #conv1
        encrel1a = self.RELU(encconv1a)       #relu
        size1 = encrel1a.size()
        #print(size1)
        encpool1,idx1 = self.enc1(encrel1a)   #pool1
        encconv2a = self.enc2a(encpool1)      #conv2
        encrel2a = self.RELU(encconv2a)       #relu
        size2 = encrel2a.size()
        # print(size2)
        encpool2,idx2 = self.enc2(encrel2a)   #pool2
        encconv3a = self.enc3a(encpool2)      #conv3
        encrel3a = self.RELU(encconv3a)       #relu 
        encconv3b = self.enc3b(encrel3a)      #conv4
        encrel3b = self.RELU(encconv3b)       #relu
        encconv3c = self.enc3c(encrel3b)      #conv5
        encrel3c = self.RELU(encconv3c)       #relu
        size3 = encrel3c.size()
        # print(size3)
        encpool3,idx3 = self.enc3(encrel3c)   #pool3
        size4 = encpool3.size()
        # print(size4)
        bottleneck = self.bridge(encpool3)    #bridge conv
        size5 = bottleneck.size()
        # print(size5)
        decpool3 = self.dec3(bottleneck, idx3, output_size = size3) #unpool3
        size6 = decpool3.size()
        # print(size6)
        decconv3c = self.dec3c(decpool3)      #deconv5
        decrel3c =self.RELU(decconv3c)        #relu
        decconv3b = self.dec3b(decrel3c)      #deconv4
        decrel3b = self.RELU(decconv3b)       #relu
        decconv3a = self.dec3a(decrel3b)      #deconv3
        decrel3a = self.RELU(decconv3a)       #relu
        decpool2 = self.dec2(decrel3a, idx2, output_size = size2)  #unpool2
        size7 = decpool2.size()
        # print(size7)
        decconv2a = self.dec2a(decpool2)      #deconv2
        decrel2a = self.RELU(decconv2a)       #relu
        decpool1 = self.dec1(decrel2a, idx1, output_size = size1)  #unpool1
        size8 = decpool1.size()
        # print(size8)
        decconv1a = self.dec1a(decpool1)      #deconv1
        sizef = decconv1a.size()
        # print(sizef)
        x = self.RELU(decconv1a)              #relu
        return x

AlexEnc_Dec = AlexEnc_Dec()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch, os
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torch.autograd import Variable
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
%matplotlib inline

In [0]:
lr = 0.0002
max_epoch = 8
batch_size = 64
z_dim = 100
image_size = 128
g_conv_dim = 68
d_conv_dim = 63
log_step = 100
sample_step = 500
sample_num = 32


In [0]:
transform = transforms.Compose([
    #transforms.Scale(64),
    transforms.Resize((215, 175)),
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = ImageFolder("//content/datareqd", transform)
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=16, drop_last=True)

In [0]:
!unzip "/content/drive/My Drive/img_align_celeba.zip"

In [0]:
ls "/content/img_align_celeba"

In [0]:
import torch.optim as optim
import time
criterion =  torch.nn.MSELoss(reduction='sum')
start_time = time.time()
optimizer = optim.Adam(AlexEnc_Dec.parameters(), lr=0.0001)
for epoch in range(max_epoch):  # loop over the dataset multiple times
    for i, (data,val) in enumerate(data_loader):
        #print(time.time()-start_time)
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()
        print(data.size())
        #data = torch.tensor(data)

        # forward + backward + optimize
        outputs = AlexEnc_Dec(data)
        loss = criterion(outputs, data)
        print(loss)
        #if i%50 == 49:
        #  print("loss:",loss)
        #  print(i)
        #  print(epoch)
        loss.backward()
        optimizer.step()

        # print statistics

print('Finished Training')

tensor(86371016., grad_fn=<MseLossBackward>)
tensor(7964447.5000, grad_fn=<MseLossBackward>)
tensor(2351750.7500, grad_fn=<MseLossBackward>)
tensor(34164312., grad_fn=<MseLossBackward>)
tensor(2833351.5000, grad_fn=<MseLossBackward>)
tensor(1.4698e+08, grad_fn=<MseLossBackward>)
tensor(6259965.5000, grad_fn=<MseLossBackward>)
tensor(2116192.2500, grad_fn=<MseLossBackward>)
tensor(2041345.8750, grad_fn=<MseLossBackward>)
tensor(2006397.5000, grad_fn=<MseLossBackward>)
tensor(2259437., grad_fn=<MseLossBackward>)
tensor(1939852.1250, grad_fn=<MseLossBackward>)
tensor(4907718.5000, grad_fn=<MseLossBackward>)
tensor(3511765., grad_fn=<MseLossBackward>)
tensor(2280611.5000, grad_fn=<MseLossBackward>)
tensor(5432575.5000, grad_fn=<MseLossBackward>)
tensor(3282654.5000, grad_fn=<MseLossBackward>)


In [0]:
!mkdir datareqd